# Stochastic Nonparametric Envelopment of Data (StoNED)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : June 13, 2020

References:

[1] Kuosmanen, T., Johnson, A. and Saastamoinen, A. (2015). Stochastic Nonparametric Approach to Efficiency Analysis: A unified Framework, in Zhu, J. (ed.) Data Envelopment Analysis. Springer, pp. 191–244.

[2] Johnson, A. L. and Kuosmanen, T. (2015). An Introduction to CNLS and StoNED Methods for Efficiency Analysis: Economic Insights and Computational Aspects, in Ray, S. C., Kumbhakar, S. C., and Dua, P. (eds) Benchmarking for Performance Evaluation: A Production Frontier Approach. Springer, pp. 117–186.

### Objective: Estimating technicial inefficiency with Quasi-likelihood Estimation (QLE)

In [1]:
# import package pystoned
from pystoned import *

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03


In [4]:
# output (total cost)
y  = df['TOTEX']

# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)

### Stage 1: Convex Nonparametric Least Squares

In [5]:
# define and solve the CNLS model

cet = "mult"
fun = "cost"
rts = "crs"

model = CNLS.cnls(y, x, cet, fun, rts)

# using remote solver (NEOS)
from pyomo.environ import SolverManagerFactory
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [6]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

eps

array([ 0.03591705,  0.02603548,  0.20219662, -0.00451797,  0.17430052,
       -0.11912249, -0.01130134,  0.1192172 ,  0.37369844, -0.04018423,
        0.02303955, -0.07779539,  0.24790125,  0.25962007, -0.02513006,
       -0.14458385,  0.19495398,  0.16927398, -0.01384278, -0.09871993,
       -0.12696537, -0.26051316,  0.18701396, -0.09403671,  0.12152759,
        0.06493156,  0.24369437, -0.19088083,  0.10527361,  0.0277184 ,
       -0.00155868, -0.43113541, -0.0450978 ,  0.21595499, -0.00401978,
        0.15996073, -0.21593312, -0.15266743, -0.115049  , -0.03704671,
       -0.04825153,  0.04513426,  0.06501279, -0.06184066, -0.08923651,
       -0.13083339,  0.02732435,  0.08403226, -0.13731314, -0.13258987,
        0.19606224, -0.02272717,  0.20040745,  0.1998938 ,  0.04287153,
       -0.21797092, -0.22973008, -0.08861547, -0.19033982,  0.02023355,
       -0.21417276, -0.08037898, -0.03575945,  0.01552399,  0.36593916,
       -0.10950723,  0.08123884, -0.04885468,  0.08217855, -0.23

### Stage 2: Residuals decomposition

In [7]:
# decomposition by method od moment (MoM)
method      = 'QLE'
res         = StoNED.stoned(y, eps, fun, method, cet)

In [8]:
# display technical inefficiency
TE = res[1]
TE

array([0.97774567, 0.98381847, 0.88100076, 1.00283498, 0.89653581,
       1.07749875, 1.00710657, 0.92802025, 0.79123508, 1.02549919,
       0.98566708, 1.04995441, 0.85612798, 0.84986443, 1.01587117,
       1.09482718, 0.88500805, 0.89936403, 1.00871164, 1.06381138,
       1.082807  , 1.17731692, 0.88942214, 1.06069418, 0.92667773,
       0.96013033, 0.85838775, 1.12705304, 0.93616396, 0.98278156,
       1.00097715, 1.31016132, 1.02866142, 0.87343828, 1.00252197,
       0.90462779, 1.1448849 , 1.10038675, 1.07475198, 1.02348507,
       1.03069621, 0.97211495, 0.96008147, 1.03951003, 1.05750861,
       1.08543459, 0.98302425, 0.94870747, 1.08985066, 1.0866299 ,
       0.88439368, 1.01434277, 0.881989  , 0.88227292, 0.97349423,
       1.14634773, 1.15482557, 1.05709716, 1.12667104, 0.98740165,
       1.14362274, 1.05165554, 1.02265986, 0.99031979, 0.79509141,
       1.0710264 , 0.95036951, 1.03108582, 0.94981007, 1.1617917 ,
       1.03097354, 0.95040278, 1.12846785, 1.06222333, 1.05651